In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm as cmap
from inspect import getargspec
import numpy as np

from wind import windmodels
from Utilities import metutils
from Utilities.maputils import bearing2theta, makeGrid, meshLatLon, gridLatLonDist
from PlotInterface.maps import saveWindfieldMap

from ipywidgets import interact, fixed, FloatSlider
import ipywidgets as widgets

import seaborn as sns
sns.set_style('ticks', {'image.cmap':'coolwarm'})
sns.set_context('poster')

profileType = 'powell'
windFieldType = 'kepert'

In [3]:
def polarGridAroundEye(lon, lat, margin=3, resolution=0.01):
    R, theta = makeGrid(lon, lat, margin, resolution)
    return R, theta

def meshGrid(lon, lat, margin=3, resolution=0.01):
    xgrid, ygrid = meshLatLon(lon, lat, margin, resolution)
    return xgrid, ygrid

In [4]:
def localWindField(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                   profileType='powell', windFieldType='kepert'):
    thetaMax = 70.
    cls = windmodels.profile(profileType)
    profile = cls(lat, lon, pEnv, pCentre, rMax*1000)
    R, theta = polarGridAroundEye(lon, lat)
    
    cls = windmodels.field(windFieldType)
    windfield = cls(profile)
    
    Ux, Vy = windfield.field(R*1000, theta, vFm, thetaFm, thetaMax)
    return Ux, Vy

In [7]:
def plotWindfield(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                  profileType='powell', windFieldType='kepert'):
    xgrid, ygrid = meshGrid(lon, lat)
    #xkm, ykm = gridLatLonDist(lon, lat, xgrid[:,0], ygrid[0,:])
    
    Ux, Vy = localWindField(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                            profileType=profileType, windFieldType=windFieldType)
    cx = rMax*np.sin(np.arange(0.00,6.29,0.01))  # Circle for rmw plots.
    cy = rMax*np.cos(np.arange(0.00,6.29,0.01))
    
    xlims=[lon-2, lon+2]
    ylims=[lat-2, lat+2]
    xkmlims = [-150, 150]
    ykmlims = [-150, 150]
    fig, axes = plt.subplots(2, 2,subplot_kw={'aspect':'equal'}, figsize=(16, 16))
    ax = axes.flatten()
    ax[0].hold(True)
    #set(plt.gca(),'DataAspectRatio',[1,1,1])
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[0].contourf(xgrid, ygrid, Ux, np.arange(-100., 101, 5))
    cs = ax[0].contour(xgrid, ygrid, Ux, np.arange(-100, 101, 5), colors='k')
    ax[0].clabel(cs, fontsize='x-small', fmt='%1.2f') #, range(-20,11,4))
    plt.colorbar(cm, ax=ax[0], orientation='horizontal')
    ax[0].plot(cx,cy,'w')
    
    ax[1].hold(True)
    #set(plt.gca(),'DataAspectRatio',[1,1,1])
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[1].contourf(xgrid, ygrid, Vy, np.arange(-100., 101, 5))
    cs = ax[1].contour(xgrid, ygrid, Vy, np.arange(-100, 101, 5), colors='k')
    ax[1].clabel(cs, fontsize='x-small', fmt='%1.2f') #, range(-20,11,4))
    plt.colorbar(cm, ax=ax[1], orientation='horizontal')
    ax[1].plot(cx,cy,'w')
    
    ax[2].hold(True)
    #set(plt.gca(),'DataAspectRatio',[1,1,1])
    ax[2].set_xlim(xlims)
    ax[2].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[2].contourf(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0., 101, 5))
    cs = ax[2].contour(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0, 101, 5), colors='k')
    ax[2].clabel(cs, fontsize='x-small', fmt='%1.2f') #, range(-20,11,4))
    plt.colorbar(cm, ax=ax[2], orientation='horizontal')
    ax[2].plot(cx,cy,'w')    
    
    ax[3].hold(True)
    ax[3].set_xlim(xlims)
    ax[3].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[3].contourf(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0., 101, 5))
    ax[3].streamplot(xgrid, ygrid, Ux, Vy, density=3, color='k', linewidth=1)
    plt.colorbar(cm, ax=ax[3], orientation='horizontal')
    fig.tight_layout()
    plt.show()


In [8]:
interact(plotWindfield, 
         lon=fixed(120),
         lat=(-20, 20, 0.1), 
         pEnv=(99500, 100000, 100),
         pCentre=(90000, 99000, 100),
         rMax=(10, 100, 1),
         vFm=(0,30, 1),
         thetaFm=(0, 2.*np.pi, 0.1),
         beta=(1.2, 2.5, 0.05))

<function __main__.plotWindfield>